# Proyecto 1: Regresión Lineal

# Parte 1: Preparación y Exploracion

In [ ]:
# carga de libererías:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

### Carga del dataset. 

In [ ]:
# Cargamos el dataset
df = pd.read_csv('dataset_proyecto_regresion.csv')

### EDA básica y Detección de outliers: 

In [ ]:
# --- 2. EDA Básica --- 

print("\n--- EDA Básica ---")

# a. Distribución y tipos de variables
print("\nInformación general del DataFrame:")
df.info()

print("\nPrimeras 5 filas del DataFrame:")
print(df.head())

print("\nEstadísticas descriptivas de las variables numéricas:")
print(df.describe())

# b. Valores faltantes
print("\nValores faltantes por columna:")
print(df.isnull().sum())
print("\nPorcentaje de valores faltantes por columna:")
print((df.isnull().sum() / len(df)) * 100)

# c. Distribución de variables categóricas
print("\nDistribución de variables categóricas:")
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    print(f"\nColumna: {col}")
    print(df[col].value_counts())
    print(f"Número de categorías únicas: {df[col].nunique()}")


# Visualización de distribuciones para variables numéricas clave
numeric_cols_for_hist = ['Age', 'AnnualIncome', 'HoursPerWeek', 'CommuteDistance', 'CreditScore', 'Debt', 'Savings', 'InternetUsage', 'TargetSpend']
plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols_for_hist):
    plt.subplot(3, 3, i + 1)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribución de {col}')
plt.tight_layout()
plt.show()

# Visualización para variables categóricas importantes 
plt.figure(figsize=(10, 6))
sns.boxplot(x='EducationLevel', y='TargetSpend', data=df)
plt.title('TargetSpend vs. EducationLevel')
plt.xticks(rotation=45)
plt.show()

### Manejo de valores faltantes: imputación categórica y numérica. 

In [ ]:
# --- Manejo de Valores Faltantes: Imputación Categórica y Numérica ---

# Imputación de valores faltantes para variables categóricas (usando la moda)
# Columnas identificadas con NaN en el EDA:
# 'MaritalStatus': 305 NaN
# 'Employed': 96 NaN
# 'OwnsHouse': 74 NaN
# 'EducationLevel': 25 NaN
# 'Gender': 32 NaN
categorical_cols_to_impute = ['MaritalStatus', 'Employed', 'OwnsHouse', 'EducationLevel', 'Gender']

for col in categorical_cols_to_impute:
    if df[col].isnull().any(): # Solo imputar si hay NaNs en la columna
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)

# Imputación de valores faltantes para variables numéricas (usando la mediana)
# Columnas identificadas con NaN en el EDA:
# 'Age': 45 NaN
# 'AnnualIncome': 45 NaN
# 'CreditScore': 45 NaN
# 'Debt': 45 NaN
# 'Savings': 45 NaN
numeric_cols_to_impute = ['Age', 'AnnualIncome', 'CreditScore', 'Debt', 'Savings']

for col in numeric_cols_to_impute:
    if df[col].isnull().any(): # Solo imputar si hay NaNs en la columna
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

# Confirmar que no hay valores faltantes después de la imputación
print("Valores faltantes después de la imputación:")
print(df.isnull().sum())

print("\nPrimeras 5 filas del DataFrame después de la imputación:")
print(df.head())

### Conversión de fechas: extraer características como AñosDesdeRegistro.

In [ ]:
# --- Manejo de Fechas: Conversión y Extracción de Características ---

# Convertir 'RegistrationDate' a formato datetime
df['RegistrationDate'] = pd.to_datetime(df['RegistrationDate'], errors='coerce')

# Calcular la antigüedad del registro en años
# Usamos la fecha actual (June 6, 2025) como referencia para 'AñosDesdeRegistro'.
# Esto asegura consistencia en el cálculo de la antigüedad.
reference_date = pd.Timestamp('2025-06-06') # Fecha de referencia explícita
df['YearsSinceRegistration'] = (reference_date - df['RegistrationDate']).dt.days / 365.25

# Extraer otras características de la fecha
# Por ejemplo, el mes de registro podría capturar estacionalidad en el gasto.
df['RegistrationMonth'] = df['RegistrationDate'].dt.month
# df['RegistrationDayOfWeek'] = df['RegistrationDate'].dt.dayofweek # 0=Lunes, 6=Domingo

# Eliminar la columna 'RegistrationDate' original
df = df.drop('RegistrationDate', axis=1)

print("\nPrimeras 5 filas del DataFrame después del manejo de fechas:")
print(df.head())

print("\nTipos de datos del DataFrame después del manejo de fechas:")
df.info()

# Parte 2: Ingeniería de Características

### Encoding de variables categóricas: OneHotEncoding y OrdinalEncoding.

In [ ]:
# Columnas categóricas nominales
categorical_nominal_cols = ['EducationLevel', 'Gender', 'MaritalStatus', 'Employed', 'JobTitle', 'OwnsHouse', 'HasPet']

# Columnas categóricas ordinales
ordinal_cols = ['FitnessLevel', 'EnvironmentalAwareness']


print("--- Categorías y conteos para variables Categóricas Nominales ---")
for col in categorical_nominal_cols:
    print(f"\nColumna: '{col}'")
    print(df[col].value_counts())
    print(f"Número de categorías únicas: {df[col].nunique()}")

print("\n--- Categorías y conteos para variables Categóricas Ordinales ---")
for col in ordinal_cols:
    print(f"\nColumna: '{col}'")
    print(df[col].value_counts())
    print(f"Número de categorías únicas: {df[col].nunique()}")


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# --- Encoding de Variables Categóricas ---

# Codificación One-Hot para variables nominales
categorical_nominal_cols = ['EducationLevel', 'Gender', 'MaritalStatus', 'Employed', 'JobTitle', 'OwnsHouse', 'HasPet']
df = pd.get_dummies(df, columns=categorical_nominal_cols, drop_first=True)

# Codificación Ordinal para variables ordinales
ordinal_cols = ['FitnessLevel', 'EnvironmentalAwareness']
# Asegurar que estas columnas sean de tipo 'category' con un orden explícito si no lo están ya
# Esto es redundante si ya se hizo, pero seguro si el estado de df no es consistente
df['FitnessLevel'] = pd.Categorical(df['FitnessLevel'], categories=[1, 2, 3, 4, 5], ordered=True)
df['EnvironmentalAwareness'] = pd.Categorical(df['EnvironmentalAwareness'], categories=[1, 2, 3, 4, 5], ordered=True)

# El encoder usará el orden especificado: [1, 2, 3, 4, 5]
encoder = OrdinalEncoder(categories=[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5]])
df[ordinal_cols] = encoder.fit_transform(df[ordinal_cols])


print("\nPrimeras 5 filas del DataFrame después del encoding:")
print(df.head())

print("\nTipos de datos del DataFrame después del encoding:")
df.info()

### Escalado: MinMaxScaler o StandardScaler. 

In [ ]:
# --- Escalado de Características Numéricas ---

# Identificar todas las columnas numéricas para escalar, excluyendo la variable objetivo 'TargetSpend'
numeric_cols_to_scale = df.select_dtypes(include=np.number).columns.tolist()
if 'TargetSpend' in numeric_cols_to_scale:
    numeric_cols_to_scale.remove('TargetSpend')

scaler = StandardScaler()
df[numeric_cols_to_scale] = scaler.fit_transform(df[numeric_cols_to_scale])


print("\nPrimeras 5 filas del DataFrame después del escalado:")
print(df.head())

print("\nTipos de datos del DataFrame después del escalado:")
df.info()

### Discretización (binning): aplicar a Age, Debt, o CreditScore.

In [ ]:
# --- Discretización (Binning) en 'Age' ---

# Discretización de 'Age' usando cuantiles (bins de igual frecuencia)
df['Age_Group'] = pd.qcut(df['Age'], q=5, labels=['Grupo_1', 'Grupo_2', 'Grupo_3', 'Grupo_4', 'Grupo_5'], precision=0)

# La nueva columna 'Age_Group' es de tipo categórico.
# Para su uso en modelos lineales, típicamente se aplicaría One-Hot Encoding a 'Age_Group' también.

print("\nPrimeras 5 filas del DataFrame después de la discretización de Age:")
print(df.head())

print("\nConteo de categorías en 'Age_Group':")
print(df['Age_Group'].value_counts())

print("\nTipos de datos del DataFrame después de la discretización:")
df.info()

In [ ]:
from sklearn.preprocessing import  PolynomialFeatures
# --- Transformaciones Polinómicas ---

# Seleccionar características para las transformaciones polinómicas.
# Elegimos 'Age' y 'AnnualIncome' ya que son variables clave 
features_for_polynomial = ['Age', 'AnnualIncome']

# Crear objeto PolynomialFeatures con grado 2 (para términos cuadrados e interacciones)
poly = PolynomialFeatures(degree=2, include_bias=False)

# Transformar las características seleccionadas
# Esto creará un array numpy con las nuevas características polinómicas
poly_features = poly.fit_transform(df[features_for_polynomial])

# Obtener los nombres de las nuevas características generadas
poly_feature_names = poly.get_feature_names_out(features_for_polynomial)

# Crear un DataFrame con las nuevas características polinómicas
df_poly = pd.DataFrame(poly_features, columns=poly_feature_names, index=df.index)

# Concatenar las nuevas características polinómicas al DataFrame original
df = pd.concat([df, df_poly], axis=1)


print("\nPrimeras 5 filas del DataFrame después de las transformaciones polinómicas:")
print(df.head())

print("\nTipos de datos del DataFrame después de las transformaciones polinómicas:")
df.info()

# Parte 3: Modelado 

### División de datos: entrenamiento y prueba (80/20). 

In [ ]:
# --- División de Datos: Entrenamiento y Prueba (80/20) ---

# Separar la variable objetivo (y) de las características (X)
X = df.drop('TargetSpend', axis=1)
y = df['TargetSpend']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nDimensiones de los conjuntos de datos:")
print(f"X_train (entrenamiento de características): {X_train.shape}")
print(f"y_train (entrenamiento de objetivo): {y_train.shape}")
print(f"X_test (prueba de características): {X_test.shape}")
print(f"y_test (prueba de objetivo): {y_test.shape}")

### Construcción de pipelines personalizados con BaseEstimator y TransformerMixin. 